In [1]:
using DataFrames, CSV, Dates, Plots, StatsPlots, MLDataPattern, MLBase, StatsPlots,Combinatorics, Optim,
DataTables, PyCall, Distributions, SpecialFunctions

In [2]:
using NBInclude
@nbinclude("Functions.ipynb")

transf_gamma_mod (generic function with 1 method)

In [3]:
#1161 et 1211 ne convergent pas 
data_files = readdir(raw"Données\data_all_OBXL_6_sites")[1:end]

51-element Vector{String}:
 "data_1011_daily.csv"
 "data_1021_daily.csv"
 "data_1031_daily.csv"
 "data_1041_daily.csv"
 "data_1051_daily.csv"
 "data_1061_daily.csv"
 "data_1071_daily.csv"
 "data_1081_daily.csv"
 "data_1091_daily.csv"
 "data_1101_daily.csv"
 "data_1111_daily.csv"
 "data_1121_daily.csv"
 "data_1131_daily.csv"
 ⋮
 "data_1491_daily.csv"
 "data_1501_daily.csv"
 "data_1521_daily.csv"
 "data_1531_daily.csv"
 "data_1541_daily.csv"
 "data_1551_daily.csv"
 "data_1561_daily.csv"
 "data_1571_daily.csv"
 "data_1581_daily.csv"
 "data_1591_daily.csv"
 "data_1601_daily.csv"
 "data_1611_daily.csv"

In [4]:
varNames = String.([:d15min, :d30min, :d45min, :d1h, :d2h, :d3h, :d4h, :d6h, :d8h, :d12h, :d24h])
toPredict = "CSO"
result_folder = raw"Résultats\All OBXL\\"

"C:\\Users\\takum\\Downloads\\Recherche\\TakumiTherville\\AAA - Final(&) scripts\\Résultats\\All OBXL\\"

# Calcul du meilleur modèle (sur tous les pluviomètres et choix de variables) pour chaque ouvrage de surverse

In [5]:
site_files = readdir(raw"Données\data_all_sites_all_OBXL")[1:end]

174-element Vector{String}:
 "data_0642-01D_daily_all_OBXL.csv"
 "data_0672-01D_daily_all_OBXL.csv"
 "data_0672-02D_daily_all_OBXL.csv"
 "data_0672-03D_daily_all_OBXL.csv"
 "data_0801-01D_daily_all_OBXL.csv"
 "data_0801-02D_daily_all_OBXL.csv"
 "data_0801-03D_daily_all_OBXL.csv"
 "data_0801-04D_daily_all_OBXL.csv"
 "data_0801-05D_daily_all_OBXL.csv"
 "data_0801-06D_daily_all_OBXL.csv"
 "data_0801-07D_daily_all_OBXL.csv"
 "data_0801-08D_daily_all_OBXL.csv"
 "data_0801-09D_daily_all_OBXL.csv"
 ⋮
 "data_4620-05D_daily_all_OBXL.csv"
 "data_4620-06D_daily_all_OBXL.csv"
 "data_4620-07D_daily_all_OBXL.csv"
 "data_4620-08D_daily_all_OBXL.csv"
 "data_4630-01D_daily_all_OBXL.csv"
 "data_4710-01D_daily_all_OBXL.csv"
 "data_4710-02D_daily_all_OBXL.csv"
 "data_4720-01D_daily_all_OBXL.csv"
 "data_4760-01D_daily_all_OBXL.csv"
 "data_4770-01D_daily_all_OBXL.csv"
 "data_4790-01D_daily_all_OBXL.csv"
 "data_4795-01D_daily_all_OBXL.csv"

In [ ]:
best_models = DataFrame()
#best_models = CSV.read(result_folder * " CV moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles 15 min.csv", DataFrame)

for filename in site_files
    
    site_name = filename[6:13]
    println(site_name)
    
    #if(!(site_name in best_models.Site)&&(site_name in ["4380-01D", "4400-01D", "4400-02D", "4420-01D", "4420-04D", "4795-01D"]))
    
        data = CSV.read(raw"Données\data_all_sites_all_OBXL\\" *filename, DataFrame)
        sort!(data, [:Date, :OBXL])
    
        if(!(size(data)[1] == 0))
            this_site_models = DataFrame()
    
    
            for this_pluvio in groupby(data, :OBXL)
        
                this_pluvio_models = DataFrame()
        
                OBXL_num = this_pluvio.OBXL[1]
                println(OBXL_num)
        
                for test_year in Dates.year(this_pluvio.Date[1]):Dates.year(this_pluvio.Date[end])
                    println(test_year)
    
                    train = this_pluvio[Dates.year.(this_pluvio.Date) .!= test_year, :]
                    test = this_pluvio[Dates.year.(this_pluvio.Date) .== test_year, :]
            
                    if(size(train[train.CSO .== 1, :])[1] == 0 || size(test[test.CSO .== 1, :])[1] == 0)
                        this_models = DataFrame(Variables = ["All"], F1 = [- Inf], Threshold = [1.0])
                    else
                        this_models = extensiveSearchLogistic(train, test, toPredict, varNames, 0.0)
                    end
            
                    this_models[!,:OBXL] .= OBXL_num
                    this_models[!,:Test_year] .= test_year
            
            
            
                    this_pluvio_models = vcat(this_pluvio_models, this_models)
                end
            
            
        
                this_pluvio_models = combine(groupby(this_pluvio_models, :Variables), [:F1, :Threshold] .=> mean)
                this_pluvio_models[!,:OBXL] .= OBXL_num
        
                this_site_models = vcat(this_site_models, this_pluvio_models)
        
        
        
            end
            this_site_models = DataFrame(this_site_models[argmax(this_site_models.F1_mean),:])
            this_site_models[!,:Site] .= site_name
     
            best_models = vcat(best_models, this_site_models)
        
            println(best_models)


    

            CSV.write(result_folder * " CV moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles 15 min.csv", best_models)
        end
    end
    
    
end

0642-01D
0672-01D
0672-02D
0672-03D
0801-01D
0801-02D
0801-03D
0801-04D
0801-05D
0801-06D
0801-07D
0801-08D
0801-09D
3230-01D
3240-01D
3240-02D
3240-03D
3240-04D
3240-05D
3250-01D
3250-02D
3260-01D
3270-01D
3275-01D
3275-02D
3280-01D
3280-02D
3290-01D
3305-01D
3305-02D
3305-03D
3305-04D
3305-05D
3305-06D
3305-07D
3310-01D
3320-01D
3330-01D
3340-01D
3350-01D
3350-02D
3350-03D
3350-04D
3350-05D
3350-06D
3350-07D
3350-08D
3350-09D
3350-10D
3350-11D
3350-12D
3370-01D
3380-01D
3390-01D
3400-01D
3410-01D
3410-02D
3411-01D
3450-01D
3450-02D
3460-01D
3480-01D
3480-02D
3480-03D
3480-04D
3480-05D
3480-06D
3490-01D
3490-02D
3500-01D
3500-02D
3500-03D
3500-04D
3520-01D
3530-01D
3540-01D
3540-02D
3560-01D
3580-01D
3580-02D
3750-01D
3762-01D
3765-01D
3767-01D
3768-01D
3780-01D
3781-01D
3782-01D
3790-01D
3790-02D
4210-01D
4230-01D
4230-02D
4230-03D
4230-04D
4230-05D
4230-06D
4230-07D
4230-08D
4230-09D
4230-10D
4240-01D
4240-02D
4250-01D
4260-01D
4265-01D
4270-01D
4270-02D
4280-01D
4280-02D
4290-01D
4

In [ ]:
# calcul sans cv meilleur obxl pour chaque ouvrage

In [13]:
best_models = DataFrame()
#best_models = CSV.read(result_folder * " moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", DataFrame)

for filename in site_files
    
    site_name = filename[6:13]
    println(site_name)
    
    #if(!(site_name in best_models.Site))
    
        data = CSV.read(raw"Données\data_all_sites_all_OBXL\\" *filename, DataFrame)
        sort!(data, [:Date, :OBXL])
    
    
        this_site_models = DataFrame()
    
    
        for this_pluvio in groupby(data, :OBXL)
        
            this_pluvio_models = DataFrame()
        
            OBXL_num = this_pluvio.OBXL[1]
            println(OBXL_num)
        
            test_year = Dates.year(this_pluvio.Date[end])
            #println(test_year)
    
            train = this_pluvio[Dates.year.(this_pluvio.Date) .!= test_year, :]
            test = this_pluvio[Dates.year.(this_pluvio.Date) .== test_year, :]
            
            if(size(train[train.CSO .== 1, :])[1] == 0 || size(test[test.CSO .== 1, :])[1] == 0)
                this_models = DataFrame(Variables = ["All"], F1 = [- Inf], Threshold = [1.0])
            else
                this_models = extensiveSearchLogistic(train, test, toPredict, varNames, 0.0)
            end
            
            this_models[!,:OBXL] .= OBXL_num
            this_models[!,:Test_year] .= test_year
            
            
            
            this_pluvio_models = vcat(this_pluvio_models, this_models)
        
            this_pluvio_models[!,:OBXL] .= OBXL_num
        
            this_site_models = vcat(this_site_models, this_pluvio_models)
        
        
        
        end
        this_site_models = DataFrame(this_site_models[argmax(this_site_models.F1),:])
        this_site_models[!,:Site] .= site_name
     
        best_models = vcat(best_models, this_site_models)
        
        println(best_models)


    

        CSV.write(result_folder * " moyenne Occurence regression logistique seuil entrainement tout OBXL meilleur modèles.csv", best_models)
    
    #end
    
    
end

0642-01D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
1×6 DataFrame
 Row │ Variables                          F1       Threshold  OBXL   Test_year  Site     
     │ Any                                Float64  Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…      1.0   0.381966   1021       2020  0642-01D
0672-01D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020

0801-04D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
8×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D

0801-09D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
13×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
17×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
20×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ ["d2h", "d3h", "d4h", "d12h"]         0.66129    0.133588   1121       2020  0672-02D
   4 │ ["d1h", "d3h", "d6h"]                 0.666667   0.266489   1341       2020  0672-03D
   5 │ All                                -Inf          1.0        1011       2020  0801-01D
   6 │ All                                -Inf          1.0        1011       

...

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
59×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

3460-01D
1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
61×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
63×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

1011
2020
1021
2020
1031
2020
1041
2020
1051
2020
1061
2020
1071
2020
1081
2020
1091
2020
1101
2020
1111
2020
1121
2020
1131
2020
1141
2020
1151
2020
1161
2020
1211
2020
1221
2020
1231
2020
1241
2020
1251
2020
1261
2020
1271
2020
1281
2020
1291
2020
1301
2020
1311
2020
1321
2020
1331
2020
1341
2020
1351
2020
1361
2020
1401
2020
1411
2020
1431
2020
1441
2020
1461
2020
1471
2020
1481
2020
1491
2020
1501
2020
1521
2020
1531
2020
1541
2020
1551
2020
1561
2020
1571
2020
1581
2020
1591
2020
1601
2020
1611
2020
65×6 DataFrame
 Row │ Variables                          F1           Threshold  OBXL   Test_year  Site     
     │ Any                                Float64      Float64    Int64  Int64      String   
─────┼───────────────────────────────────────────────────────────────────────────────────────
   1 │ ["d1h", "d2h", "d3h", "d4h", "d6…     1.0        0.381966   1021       2020  0642-01D
   2 │ All                                -Inf          1.0        1011       2020  0672-01D
   3 │ 

LoadError: ArgumentError: column name :F1 not found in the data frame since it has no columns

# Calcul de la distance séparant les ouvrages de surverse et les différents pluviomètres

Le calcul de la distance est effectué dans le script effectuant les mêmes calculs avec les données horaires. Celui-ci est disponible dans le même fichier que ce script au nom "Tous pluviomètres et ouvrages occurrence données horaires.ipynb"